<a href="https://colab.research.google.com/github/ahnjonghyunn/ABC/blob/main/%EC%9E%90%EB%8F%99%EC%B0%A8_%EC%97%B0%EB%B9%84(MPG)%EC%98%88%EC%B8%A1%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자동차 연비(MPG) 예측하기

* 1970년대 후반 1980년대 초반의 자동차 연비 예측

##1 데이터 준비하기(임포트)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#데이터 준비
df=pd.read_csv('/content/auto-mpg.csv',header=None) #헤더가 없는 경우
df

,0,1,2,3,4,5,6,7,8
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.00,2790.0,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.00,2130.0,24.6,82,2,vw pickup
395,32.0,4,135.0,84.00,2295.0,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.00,2625.0,18.6,82,1,ford ranger


In [ ]:
#컬럼 만드는법
df.columns = ['MPG','Cylinders','Displacement','Horsepower','Weight', #연비, 실린더, 배수량, 마력, 중력,
                'Acceleration', 'Model Year', 'Origin', 'Name']  #가속, 연식, 제조국(1;USA, 2;Europe, 3;Japan)]

In [ ]:
df

In [ ]:
dataset=df.copy()
dataset.info()

In [ ]:
dataset.isna().sum()

##2 데이터 정제하기

In [ ]:
dataset['Horsepower'].unique()

In [ ]:
# '?' 값을 제거
# '?' ->np.nan->dropna() vs fillna() ->Horsepower dtype float 변환
# 1) ?-> np.nan 형태로 변경
dataset['Horsepower'].replace('?', np.nan, inplace=True)
dataset.isna().sum()

In [ ]:
# 2) np.nan 값을 삭제
dataset.dropna(subset=['Horsepower'], axis=0, inplace=True)
dataset.isna().sum()


In [ ]:
#3) 데이터 타입(object) -> float 변경
dataset['Horsepower'] = dataset['Horsepower'].astype('float')
dataset.info()

In [ ]:
#dataset #수치형태면 그냥쓰고 문자열이있으면 인코딩 name
#수치화 시킨다 그걸 비교해야한다? 원핫 인코딩 해야함


### Origin 수치형 -> 범주형 원핫인코딩 변환

In [ ]:
origin = dataset.pop('Origin') #끄집어내기, 잘라내기
dataset['USA'] = (origin ==1) * 1.0
dataset['Europe'] = (origin ==2) * 1.0
dataset['Japan'] = (origin ==3) * 1.0

dataset.tail()

In [ ]:
#필요 없는 칼럼 삭제
del dataset['Name']
dataset.head()

##3 훈련세트와 테스트세트 분리

In [ ]:
#훈련세트(80%,314), 테스트 세트(20%, 78)
train_dataset = dataset.sample(frac=0.8,random_state=0) # 80%를 임의 추출
test_dataset = dataset.drop(train_dataset.index) # 80% 임의 추출된 데이터셋 삭제후 20% 담기


In [ ]:
# test_dataset.shape   314
# train_dataset.shape   78
# feature(특성, 독립변수, 원인. 문제집)와 label(레이블, 종속변수, 결과, 답안지) 분리
train_label = train_dataset.pop('MPG')
test_label = test_dataset.pop('MPG')


##44. 스케일링 ->표준화(Standardzation)

In [ ]:
mean =np.mean(train_dataset, axis=0)
std=np.std(train_dataset, axis=0)

train_dataset = (train_dataset-mean)/std
test_dataset = (test_dataset-mean)/std

train_dataset.head()


##5 모델 구성하기

In [ ]:
len(train_dataset.keys())

In [ ]:
len(train_dataset.columns)

In [ ]:
#연비예측
model = Sequential()
model.add(Dense(64, activation = 'relu', input_shape=(len(train_dataset.columns), )))
model.add(Dense(64, activation= 'relu'))
model.add(Dense(1)) # 연속적인 값이니까 1 수치값을 써야해서 리뉴어

##6 모델 설정하기

In [ ]:
model.compile(optimizer=RMSprop(0.001), loss = 'mse', metrics=['mae','mse']) #연속적인값은 손실값 ,정확도->이진 다중

##7 모델 학습하기

In [ ]:
history=model.fit(train_dataset,train_label,epochs=500) #손실값이 떨어지는지 봐야함

##8 성능평가

In [ ]:
import matplotlib.pyplot as plt

his_dict = history.history
mse = his_dict['mse']

epochs = range(1, len(mse) + 1)
fig = plt.figure(figsize = (10, 5))

# 훈련 및 검증 손실 그리기
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(epochs, mse, color = 'blue', label = 'train_mse')
ax1.set_title('train mse')
ax1.set_xlabel('epochs')
ax1.set_ylabel('mse')


ax1.legend()

mae = his_dict['mae']

# 훈련 및 검증 정확도 그리기
ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(epochs, mae, color = 'blue', label = 'train_mae')
ax2.set_title('train mae')
ax2.set_xlabel('epochs')
ax2.set_ylabel('mae')
ax2.legend()

plt.show()

### 모델 평가하기

In [ ]:
model.evaluate(test_dataset, test_label) #1.7 평균 절대오차 ->실제 연비와 1.7

### 모델 예측

In [ ]:
test_predictions = model.predict(test_dataset).flatten()

plt.scatter(test_label,test_predictions) # x축 정답 y축 예측한값

plt.scatter(test_label, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])